# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import datetime
import pandas as pd

from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
df = cda.crash_dataframe()

In [4]:
lat = 39.720934
lon = -104.959436

In [9]:
nearby = cda.crashes_near_point(
    df[df.reported_date.dt.date == datetime.date(2022,12,14)]
    , lat
    , lon
    , radius_miles=1
)

In [10]:
nearby

,incident_id,top_traffic_accident_offense,reported_date,incident_address,geo_lon,geo_lat,neighborhood_id,bicycle_ind,pedestrian_ind,SERIOUSLY_INJURED,...,fatality,sbi_or_fatality,crash_date_str,crash_time_str,crash_year,crash_day_of_year,one,target_lat,target_lon,distance_miles
22839,2022639693,TRAF - ACCIDENT - HIT & RUN,2022-12-14 13:00:00-07:00,E 10TH AVE / N LAFAYETTE ST,-104.970639,39.732046,Cheesman Park,0.0,0.0,NaN,...,False,False,2022-12-14 Wed,"Wed Dec 14, 1:00 PM",2022,348,1,39.720934,-104.959436,0.971498
22359,2022639830,TRAF - ACCIDENT,2022-12-14 15:02:00-07:00,E 3RD AVE / N UNIVERSITY BLVD,-104.959436,39.720981,Country Club,0.0,1.0,0.0,...,False,False,2022-12-14 Wed,"Wed Dec 14, 3:02 PM",2022,348,1,39.720934,-104.959436,0.003213


In [11]:
df[df.incident_id == 2022639830].squeeze()

incident_id                                         2022639830
top_traffic_accident_offense    TRAF - ACCIDENT               
reported_date                        2022-12-14 15:02:00-07:00
incident_address                 E 3RD AVE / N UNIVERSITY BLVD
geo_lon                                            -104.959436
geo_lat                                              39.720981
neighborhood_id                                   Country Club
bicycle_ind                                                0.0
pedestrian_ind                                             1.0
SERIOUSLY_INJURED                                          0.0
FATALITIES                                                 0.0
sbi                                                      False
fatality                                                 False
sbi_or_fatality                                          False
crash_date_str                                  2022-12-14 Wed
crash_time_str                             Wed Dec 14, 

In [8]:
len(df[df.reported_date.dt.date == datetime.date(2022,12,14)])

51